In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [56]:
class GetThreads():
    def __init__(self, page_num, find_threads, find_authors, find_replies, find_views, find_titles):
        self.page_num = page_num
        self.find_threads = find_threads
        self.find_authors = find_authors
        self.find_replies = find_replies
        self.find_views = find_views
        self.find_titles = find_titles

    def isolate_author(self):
        authors = []
        for i in range(len(self.find_authors)):
            text = self.find_authors[i].getText()
            author = text.split()[0][:-2]
            authors.append(author)
        return authors
    
    def isolate_views(self):
        views = []
        for i in range(len(self.find_views)):
            total_views = int(self.find_views[i].getText().replace(" ",'').replace(',','')[5:-4])
            views.append(total_views)
        return views
    
    def isolate_replies(self):
        replies = []
        for i in range(len(self.find_replies)):
            total_replies = int(self.find_replies[i].getText().replace(" ",'').replace(',','')[5:-4])
            replies.append(total_replies)
        return replies
        
    def isolate_titles_links(self):
        titles = []
        links = []
        for i in range(len(self.find_titles)):
            title = self.find_titles[i].find('a').getText()[34:-30]
            ahref = self.find_titles[i].find('a', href=True)['href']
            titles.append(title)
            links.append(ahref)

        return [titles, links]



In [ ]:
def scrapeThreads():
    threadsMap = {}
    for num in range(1,339):
        print(f'Page {num}')
        url = f'https://forum.bodybuilding.com/forumdisplay.php?f=19&page={num}&order=desc'
        page = requests.get(url)
        soup = BeautifulSoup(page.text,'html.parser')
        find_threads = soup.find_all('div', {'class':"threadinfo"})
        find_authors = soup.find_all('div', {'class':"author"})
        find_replies = soup.find_all('td', {'class':"othercol td-reply"})
        find_views = soup.find_all('td', {'class':"alt-col othercol td-views"})
        find_titles = soup.find_all('h3', {'class':"threadtitle"})
        misc = GetThreads(num, find_threads, find_authors, find_replies, find_views, find_titles)
        authors = misc.isolate_author()
        views = misc.isolate_views()
        replies = misc.isolate_replies()
        titles = misc.isolate_titles_links()[0]
        links = misc.isolate_titles_links()[1]
        for i in range(len(links)):
            author = authors[i]
            threadviews = views[i]
            threadreplies = replies[i]
            threadtitle = titles[i]
            threadlink = links[i]
            if threadlink in threadsMap:
                continue
            threadsMap[threadlink] = [threadtitle, author, threadviews, threadreplies]  

    return threadsMap
    

In [9]:
# threadsData = scrapeThreads()

In [348]:
# by_replies = misc_df.sort_values(by=['Replies'], ascending=False).head(100)
# by_views = misc_df.sort_values(by=['View'], ascending=False).head(100)
# unique_miscers = misc_df['Author'].unique().tolist()


In [455]:
# #views = misc_df.loc[misc_df['Author'].unique(), 'View'].sum()

# #f2 = misc_df.groupby(['Author'])['View'].count().to_frame()

# f1 = misc_df['Author'].value_counts().to_frame()
# f2 = misc_df.groupby('Author')['Replies'].sum().to_frame()
# f3 = misc_df.groupby('Author')['View'].sum().to_frame()
# f5 = f1.merge(f2, left_index=True, right_index=True).merge(f3, left_index=True, right_index=True)
# f5.to_excel('topPosters.xlsx')
